In [1]:
#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").config('spark.driver.memory', '32g').config("hive.server2.thrift.port", 10000).config("spark.sql.hive.thriftServer.singleSession", True).enableHiveSupport().getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/12 11:09:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())    
])

df1 = spark.read.format("csv").option("header", "false").schema(schema).load("file:/Users/minoseah629/Desktop/ait614/*5M/*.csv")
df1.printSchema()
# .option("inferSchema", "true")

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: string (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [7]:
df2 = df1.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df1.na.drop()
df1.createOrReplaceTempView("dataset")

df1.count()

58892322

In [8]:
year = '2017'
start = year+'-01-01'
end = year+'-12-31'
query = "select ID,Speed,TravelTime,Status,timedate,LinkId,LinkPoints,EncodedLinkPoints,EncodedPolyLineLvls,Owner,TranscomId,Borough,Link_Name from dataset"
where = "where timedate >= '"+start+"' and timedate <= '"+end+"' order by timedate asc"

In [9]:
y2017 = spark.sql(query + " "+where)

In [10]:
y2017.write.format('csv').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/year'+year+'.csv')

In [12]:
df1.createOrReplaceTempView("dataset")
linkpoints = spark.sql("select count(EncodedLinkPoints),EncodedLinkPoints from dataset where id = 148 group by EncodedLinkPoints")

In [7]:
y2021.show()

+---+-----+----------+------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+----------+-------------+--------------------+
| ID|Speed|TravelTime|Status|           timedate| LinkId|          LinkPoints|   EncodedLinkPoints| EncodedPolyLineLvls|               Owner|TranscomId|      Borough|           Link_Name|
+---+-----+----------+------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+----------+-------------+--------------------+
|190|40.38|       105|     0|2022-01-01 00:03:10|4620332|40.84671,-73.9319...|}zhxFryfbMfC_J~Bq...|           BBBBBBBBB|         PA-GWBridge|   4620332|        Bronx|CBE W MORRIS AVE ...|
|186|52.81|       157|     0|2022-01-01 00:03:10|4620315|40.84667,-73.9318...|uzhxFfyfbMcCbIsGf...|      BBBBBBBBBBBBBB|         PA-GWBridge|   4620315|        Bronx|CBE W L/LE V AMST...|
|324| 8.69|       627|  -101|2022-01-01 00:03:10|4329473|40.

In [36]:
df1.createOrReplaceTempView("noschema")
spark.sql("select max(length(_c6)) from noschema").show()

+----------------+
|max(length(_c6))|
+----------------+
|             255|
+----------------+



In [11]:
spark.stop()